### 1) Creat a new database in PostgreSQL

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "entity_phone"
#os.environ['SNORKELDB']
os.environ['SNORKELDBNAME'] = "location_extraction"
print os.environ['SNORKELDBNAME']
os.environ['SNORKELDB'] = 'postgresql://localhost:5432/' + os.environ['SNORKELDBNAME']
#from sqlalchemy import create_engine
#snorkeldb = create_engine('postgresql://localhost:5432/', isolation_level="AUTOCOMMIT")


sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/fonduer/memex/')

location_extraction


 ## 1.1 Defining a Candidate Schema

In [2]:
from snorkel.contrib.fonduer import SnorkelSession

session = SnorkelSession()

In [3]:
import os
from snorkel.contrib.fonduer.models import candidate_subclass

Location_Extraction = candidate_subclass('location_extraction', ["location"])


## 1.2 Parsing and Transforming the Input Documents into Unified Data Models

### Configuring an `HTMLPreprocessor`

In [4]:
from snorkel.contrib.fonduer import HTMLPreprocessor, OmniParser

docs_path = os.environ['SNORKELHOME'] + '/tutorials/fonduer/memex/data/texas_profiles_data/'

doc_preprocessor = HTMLPreprocessor(docs_path)

### Configuring an `OmniParser`

In [5]:
corpus_parser = OmniParser(structural=True, lingual=True)
%time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

Clearing existing...
Running UDF...
CPU times: user 8.86 s, sys: 153 ms, total: 9.01 s
Wall time: 4min 38s


In [6]:
from snorkel.contrib.fonduer.models import Document, Phrase,Table

print "Documents:", session.query(Document).count()
print "Phrases:", session.query(Phrase).count()
print "Table", session.query(Table).count()


Documents: 342
Phrases: 71150
Table 832


## 1.3 Dividing the Corpus into Test and Train

In [6]:
docs = session.query(Document).order_by(Document.name).all()
ld   = len(docs)

train_docs = set()
dev_docs   = set()
test_docs  = set()
splits = (0.8, 0.9)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)
from pprint import pprint
#pprint([x.name for x in train_docs])
print "train:",len(train_docs)
print "dev:" ,len(dev_docs)
print "test:",len(test_docs)
# from pprint import pprint
# pprint([x.name for x in train_docs])

train: 274
dev: 34
test: 34


### Phase 2: Candidate Extraction & Multimodal Featurization

In [7]:
from snorkel.matchers import *
location_matcher = LocationMatcher(longest_match_only=True) 

####Define a relation's ContextSpaces

from snorkel.contrib.fonduer.fonduer.candidates import OmniNgrams
location_ngrams = OmniNgrams(n_max=6, split_tokens=[])


### Defining candidate Throttlers

In [8]:
from snorkel.contrib.fonduer.lf_helpers import *
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)


    
def location_currencies_filter(location):
    list_currencies = [ "dollar", "dollars", "lira","kwacha","rials","rial","dong","dongs","fuerte","euro",
                       "euros","vatu","som","peso","sterling","sterlings","soms","pestos",
                       "pounds", 
                  "pound","dirham","dirhams","hryvnia","manat","manats","liras","lira",
                       "dinar","dinars","pa'anga","franc","baht","schilling",
                  "somoni","krona","lilangeni","rupee","rand","shilling","leone","riyal","dobra",
                  "tala","ruble","zloty","peso","sol","quarani","kina","guinean","balboa","krone","naira",
                  "cordoba","kyat","metical","togrog","leu","ouguiya","rufiyaa","ringgit","kwacha",
                  "ariary","denar","litas","loti","lats","kip","som","won","tenge","yen","shekel","rupiah",
                  "forint","lempira","gourde","quetzal","cedi","lari","dalasi","cfp","birr","kroon","nakfa",
                  "cfa","Peso","koruna","croatian","colon","yuan","escudo","cape","riel","lev","real"
                  ,"real","mark","boliviano","ngultrum","taka","manat","dram","kwanza","lek","afghani","renminbi"]

    
    cand_right_tokens = list(get_right_ngrams(location,window=2))
    #print len(cand_right_tokens)
    #print cand_right_tokens#(get_right_ngrams(location,window=4))
    for cand in cand_right_tokens:
        #print "["+cand+"]"
        if cand not in list_currencies:
            #print "["+cand+"]"
            #print location
            return location
    
candidate_filter = location_currencies_filter

In [9]:
from snorkel.contrib.fonduer.candidates import CandidateExtractor

candidate_extractor = CandidateExtractor(Location_Extraction,
                                         [location_ngrams], [location_matcher],
                                         candidate_filter=candidate_filter)


#                         candidate_filter=candidate_filter

%time candidate_extractor.apply(train_docs, split=0, parallelism=PARALLEL)

Clearing existing...
Running UDF...
CPU times: user 82.5 ms, sys: 37.4 ms, total: 120 ms
Wall time: 12.2 s


### Repeating for the test and dev set

In [10]:
%%time
for i, docs in enumerate([dev_docs, test_docs]):
    candidate_extractor.apply(docs, split=i+1)
    print "Number of candidates:", session.query(Location_Extraction).filter(Location_Extraction.split == i+1).count()
    
    

Clearing existing...
Running UDF...
[========================================] 100%
Number of candidates: 263
Clearing existing...
Running UDF...
[========================================] 100%
Number of candidates: 269
CPU times: user 6.05 s, sys: 265 ms, total: 6.31 s
Wall time: 7.87 s


### Create an .xlxs for the locations and document name


In [48]:
'''create an .xlxs from location candidate
    convert all words to lowercase and also change united states to usa'''

import xlsxwriter
all_cands = session.query(Location_Extraction).all()
c_1 = all_cands[1]

x=c_1.get_parent()
y=str(x)

def doc_name_extraction(inputstr):
    p = inputstr.get_parent()
    y = str(p)
    return y.split("(")[1].split(",")[0].split(": ")[1]

def location(data):
    return data[0].get_span()

# doc_name_extraction(y)
dict_cands ={}

for c in all_cands:
    file_name = doc_name_extraction(c)
    curr_loc = location(c)
    if curr_loc =="United States":
        curr_loc = "USA"
    if file_name in dict_cands.keys():
        dict_cands[file_name].append(curr_loc.lower())
        dict_cands[file_name] =list(set( dict_cands[file_name]))
    else:
        
        dict_cands[file_name] = [curr_loc.lower()]

workbook = xlsxwriter.Workbook('texas_locations_profiles_ver4.xlsx')
worksheet = workbook.add_worksheet()

d = dict_cands
row = 0
col = 0
worksheet.write(0, 0, "file name")
worksheet.write(0, 1, "location_pred")
for key in d.keys():
    row += 1
    worksheet.write(row, col, key)
    
    worksheet.write(row, col + 1, ','.join(map(str,list(set(d[key])))))
workbook.close()
        

In [10]:
all_cands[1].get_parent().document.name

u'760e158f-4de8-4311-869e-50ccb9f6b237'

### Repeating for development and test splits

In [14]:
%%time
for i, docs in enumerate([dev_docs, test_docs]):
    candidate_extractor.apply(docs, split=i+1)
    print "Number of candidates:", session.query(Location_Extraction).filter(Location_Extraction.split == i+1).count()

 Clearing existing...
Running UDF...
[========================================] 100%
Number of candidates: 263
Clearing existing...
Running UDF...
[========================================] 100%
Number of candidates: 269
CPU times: user 9.28 s, sys: 542 ms, total: 9.82 s
Wall time: 15.4 s


In [ ]:
session.rollback()

In [ ]:
# dev_cands = session.query(Location_Extraction).filter(Location_Extraction.split == 1).all()
# print "Number of candidates:", len(train_cands)
# dev_cand1= dev_cands[300]
# # for cand in dev_cand:
# #     print cand
# #     print cand.get_parent()
# print get_ancestor_tag_names(dev_cand1)
# print dev_cand1.get_parent()

### Create a markup file for the locations and document name, 

In [5]:
'''create an .xlxs from location candidate
    convert all words to lowercase and also change united states to usa'''
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)
from snorkel.contrib.fonduer.lf_helpers import *
import xlsxwriter
import csv
import itertools
all_cands = session.query(Location_Extraction).all()
c_1 = all_cands[1]
print len(all_cands)
x=c_1.get_parent()
y=str(x)

def doc_name_extraction(c):
    
    return c.get_parent().document.name
#     p = inputstr.get_parent()
#     y = str(p)
#     return y.split("(")[1].split(",")[0].split(": ")[1]

def extracted_candidate(c):
#     return data[0].get_span()
    return c.location.get_span().encode('ascii','ignore')
def extracted_text_list(c):
    return c.get_parent().text.encode('ascii','ignore')
    #return c.get_parent().words.encode('ascii','ignore')
# def candidate_right_token(c):
#     x=list(get_right_tokens(c,window=40))
#     lst=[]
#     for word in x:
#         lst.append(word.encode('ascii','ignore'))
#     return lst
# def candidate_left_token(c):
#     x=list(get_left_tokens(c,window=40))
    
#     lst=[]
#     for word in x:
#         word=  word.encode('ascii','ignore')
#         lst.append(word)
#     return lst
def candidate_neighbor_phrase(c):
    x=list(get_neighbor_phrase_ngrams(c, d=4, attrib='words', n_min=1, n_max=1, lower=True))
    lst=[]
    for word in x:
        lst.append(word.encode('ascii','ignore'))
    return lst
def candidate_xpath(c):
    x =c[0].get_parent().xpath.encode('ascii','ignore')
    #print x
    
    return x

def candidate_strating_word(c):
    return c[0].get_word_start()

def candidate_ending_word(c):
    return c[0].get_word_end()
# doc_name_extraction(y)

# def span_HTML_attribute (c):
#     ll = get_attributes(c)
#     t = type(ll)
#     if t == list:
#     lst=[]
#     for l in ll:
#         lst.append(l.encode('ascii','ignore'))
#     return lst
#     else:
#         return ll.encode('ascii','ignore')
def span_HTML_attribute (c):
    ll = get_attributes(c)
    t = type(ll)
    if t == list:
        lst=[]
        for l in ll:
            lst.append(l.encode('ascii','ignore'))
        return lst
    else:
        return ll.encode('ascii','ignore')
    
result_list=list()
counter = 0
for c in all_cands:
    
    dict_cands ={}
    file_name = doc_name_extraction(c)
    cur_key = str(file_name)+"_"+ str(counter)
    curr_loc = extracted_candidate(c)
    
  #  if curr_loc =="United States":
  #      curr_loc = "USA"
#     if file_name in dict_cands.keys():
#         dict_cands[file_name].append(curr_loc.lower())
#         dict_cands[file_name] =list(set( dict_cands[file_name]))
        #print "hi"
    dict_cands["extracted_candidate"] = [curr_loc.lower()]
    
  #  else:
        
    dict_cands["extracted_candidate"] = [curr_loc.lower()]
    dict_cands["file_name"] = file_name
    dict_cands["text_list"] = extracted_text_list(c)
    dict_cands["HTML attributes of span"]= span_HTML_attribute (c)
    dict_cands["candidate_neighbor_phrase"] = candidate_neighbor_phrase(c)
    dict_cands["started_word"] = candidate_strating_word(c)
    dict_cands["ended_word"] = candidate_ending_word(c)
    
#     dict_cands["right_token"]= candidate_right_token(c)
#     dict_cands["left_token"] = candidate_left_token(c)
    dict_cands["candidate_xpath"] = candidate_xpath(c)
    dict_cands_2 = {}
    dict_cands_2[cur_key] = dict_cands
    result_list.append(dict_cands_2)
    counter =counter+1
    

nested = result_list

headings = [d.keys()[0] for d in nested]
entries = [sorted(nested[index][col].items()) for index, col in enumerate(headings)]    
#print headings
#print "hi"
with open('output_7.csv', 'wb') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerow(['mykey'] + headings)

    for cols in itertools.izip_longest(*entries, fillvalue=['<n/a>']*len(entries[0])):
        csv_output.writerow([cols[0][0]] + [col[1] for col in cols])


from itertools import izip
from csv import reader, writer

with open('output_7.csv') as f:
    with open('destination_3.csv', 'w') as fw:
        writer(fw, delimiter=',').writerows(izip(*reader(f, delimiter=',')))

2787


### Exploring more:

https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/

In [ ]:
cand_16= train_cands[16]
p =  cand_16.get_parent()

print "text for the 16th candidate:\n", cand_16.get_parent()
#print common_ancestor(cand_16)
#print "16th candidate\n:",cand_16[0]
ance_16 = get_ancestor_tag_names(cand_16)
print "ancestor of 16th candidate\n:", ance_16 
#print ance_16.get_span()
print "***************************************************"
cand_17= train_cands[17]
print "text for the 17th candidate:\n", cand_17.get_parent()
print "17th candidate:",cand_17
ance_17 = get_ancestor_tag_names(cand_17)
print "ancestor of 17th candidate\n:", ance_17
print "***************************************************"

cand_19= train_cands[19]
print "text for the 19th candidate:\n", cand_19.get_parent()
print "19th candidate:",cand_19
print type(cand_19)
ance_19 = get_ancestor_tag_names(cand_17)
print "ancestor of 19th candidate\n:", ance_19

In [ ]:
cand_16= train_cands[16]
print cand_16

cand_18= train_cands[18]
print cand_18
cand_19= train_cands[19]
print cand_19
print cand_19[0].get_span()
get_phrase_ngrams(cand_19[0].get_span())
train_doc[19]